In [1]:
import numpy as np
import pandas as pd

In [2]:
from model import Model

In [3]:
data = pd.read_csv("penguins.csv")

In [4]:
data

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,gender,body_mass_g
0,Adelie,39.1,18.7,181,male,3750
1,Adelie,39.5,17.4,186,female,3800
2,Adelie,40.3,18.0,195,female,3250
3,Adelie,39.6,17.7,186,female,3500
4,Adelie,36.7,19.3,193,female,3450
...,...,...,...,...,...,...
145,Chinstrap,50.8,18.5,201,male,4450
146,Chinstrap,50.1,17.9,190,female,3400
147,Chinstrap,49.0,19.6,212,male,4300
148,Chinstrap,51.5,18.7,187,male,3250


In [5]:
data.fillna('pad', inplace=True)
data['gender'] = np.where(data['gender'] == 'male', 1, 0)
data['flipper_length_mm'] = data['flipper_length_mm'] / 10
data['body_mass_g'] = data['body_mass_g'] / 1000
func = lambda x: x / 100
data['flipper_length_mm'] = data['flipper_length_mm'].apply(func)
data['bill_length_mm'] = data['bill_length_mm'].apply(func)
data['bill_depth_mm'] = data['bill_depth_mm'].apply(func)
data['body_mass_g'] = data['body_mass_g'].apply(func)

In [6]:
data

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,gender,body_mass_g
0,Adelie,0.391,0.187,0.181,1,0.03750
1,Adelie,0.395,0.174,0.186,0,0.03800
2,Adelie,0.403,0.180,0.195,0,0.03250
3,Adelie,0.396,0.177,0.186,0,0.03500
4,Adelie,0.367,0.193,0.193,0,0.03450
...,...,...,...,...,...,...
145,Chinstrap,0.508,0.185,0.201,1,0.04450
146,Chinstrap,0.501,0.179,0.190,0,0.03400
147,Chinstrap,0.490,0.196,0.212,1,0.04300
148,Chinstrap,0.515,0.187,0.187,1,0.03250


In [7]:
labels = {"Adelie": 0, "Gentoo": 1, "Chinstrap": 2}

In [8]:
data["species"] = data["species"].apply(lambda x: labels[x])

In [9]:
data

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,gender,body_mass_g
0,0,0.391,0.187,0.181,1,0.03750
1,0,0.395,0.174,0.186,0,0.03800
2,0,0.403,0.180,0.195,0,0.03250
3,0,0.396,0.177,0.186,0,0.03500
4,0,0.367,0.193,0.193,0,0.03450
...,...,...,...,...,...,...
145,2,0.508,0.185,0.201,1,0.04450
146,2,0.501,0.179,0.190,0,0.03400
147,2,0.490,0.196,0.212,1,0.04300
148,2,0.515,0.187,0.187,1,0.03250


In [10]:
def partition_data(data, y_label):
    # Create empty DataFrames
    training = pd.DataFrame()
    testing = pd.DataFrame()

    # Group by the label, then add 30 training rows to the training DataFrame
    # and 20 test rows to the testing frame
    for _, group in data.groupby(y_label):
        training = pd.concat([training, group.iloc[:30]], ignore_index=True)
        testing = pd.concat([testing, group.iloc[30:]], ignore_index=True)

    # Randomly shuffle the data
    training = training.sample(frac=1)
    testing = testing.sample(frac=1)

    # Return the feature columns and label column separately for each DataFrame, such that the return values
    # are training_feature_columns, training_label_column, testing_feature_columns, testing_label_coloumn
    # Inspired by scikit-learn train_test_split function
    return (
        training.loc[:, training.columns != y_label],
        training[y_label],
        testing.loc[:, testing.columns != y_label],
        testing[y_label]
    )

In [11]:
x_train, y_train, x_test, y_test = partition_data(data, "species")

In [12]:
model = Model(
    x_train, y_train,
    x_test, y_test,
    [12, 16, 8],
    bias = 1,
    mse_threshold = 0.01
)

In [13]:
model.train()

In [14]:
model.test()

95.0